## Import Libraries

In [1]:
import pandas as pd

In [2]:
from scipy.fftpack import fft, rfft, irfft

In [3]:
from scipy import signal
import numpy as np
import math

In [4]:
from sklearn import linear_model

In [5]:
import sys

## Initializations

In [6]:
# make a list of the features, to be used for making the dataframes

# features = ['5KMax', '5KAvg', '5KStd',
#             '10KMax', '10KAvg', '10KStd',
#             '15KMax', '15KAvg', '15KStd',
#             '20KMax', '20KAvg', '20KStd',
#             '25KMax', '25KAvg', '25KStd',
#             '30KMax', '30KAvg', '30KStd']
features = []
for i in range(1, 7):
    features.append(str(i * 5) + 'KMax')
    features.append(str(i * 5) + 'KAvg')
    features.append(str(i * 5) + 'KStd')    

In [ ]:
features

## Calculate Chunks FFT

In [ ]:
def calculate_freqs_amps(values, T=1):
    s = np.array(values.iloc[:, 0])
#     t = np.array(values.iloc[:, 1])
    fft = np.fft.fft(s)
#     T = t[1] - t[0]  # sampling interval 
    N = s.size
#     N = len(s)

    # 1/T = frequency
    f = np.linspace(0, 1 / T, N)

    # frequencies are f[:N // 2]
    # amplitudes are np.abs(fft)[:N // 2] * 1 / N
    return f[:N // 2], np.abs(fft)[:N // 2] * 1 / N

In [7]:
def calculate_fft_coeffs(values):
    rfft_values = rfft(values)

#     return [rfft_values[5000], rfft_values[10000], rfft_values[15000], rfft_values[20000]]
    result = []
    for i in range(1, 7):
        result.append(np.amax(rfft_values[0:i * 5000]))
        result.append(np.mean(rfft_values[0:i * 5000]))
        result.append(np.std(rfft_values[0:i * 5000]))

#     return [np.amax(rfft_values[0:5000]),
#             np.mean(rfft_values[0:5000]),
#             np.amax(rfft_values[5000:10000]),
#             np.mean(rfft_values[5000:10000]),
#             np.amax(rfft_values[10000:15000]),
#             np.mean(rfft_values[10000:15000]),
#             np.amax(rfft_values[15000:20000]),
#             np.mean(rfft_values[15000:20000]),
#             np.amax(rfft_values[20000:25000]),
#             np.mean(rfft_values[20000:25000])
#            ]
    return result

In [ ]:
# train_set = pd.read_csv('../data/train/1.csv', float_precision='round_trip')
# train_set = pd.read_csv('../data/train/temp-small.csv', float_precision='round_trip')

## Train the Model

In [8]:
train_set_csvs = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']
# train_set_csvs = ['1', '17']
# train_set_csvs = ['1', '2', '3', '4', '5', '6', '7']
# train_set_csvs = ['1-small', '2-small', '3-small']

In [ ]:
# csv_file_name_full_path = '../data/train/' + '1-small' + '.csv'
# temp_train_set = pd.read_csv(csv_file_name_full_path, float_precision='round_trip')
# for g, df in temp_train_set.groupby(np.arange(len(temp_train_set)) // 150000):
#     print('df.iloc[:, 0]:', list(df.iloc[:, 0]))

In [9]:
# T = 1.469099981 - 1.4690999799
for csv_file_name in train_set_csvs:
    csv_file_name_full_path = '../data/train/' + csv_file_name + '.csv'
    train_set = pd.read_csv(csv_file_name_full_path, float_precision='round_trip')
    
    # initialize the DataFrame for this file
#     csv_file_stats_df = pd.DataFrame(columns=['Average', 'STD', 'Time_Remained'])
#     csv_file_stats_df = pd.DataFrame(columns=['5000', '10000', '15000', '20000', '25000', 'Time_Remained'])
    csv_file_stats_df = pd.DataFrame(columns=features+['Time_Remained'])
#     csv_file_stats_df = pd.DataFrame()

    # we take chunk_size for every training set
    # 150K is chosen for this case because the test sizes are the same
    chunk_size = 150000
    for g, df in train_set.groupby(np.arange(len(train_set)) // chunk_size):
#         print('Working on file', csv_file_name, 'chunk', str(g) + '/' + str(len(train_set) // chunk_size))
        status_text = '\rWorking on file ' + \
                      csv_file_name + \
                      ' chunk ' + \
                      str(g) + '/' + str(len(train_set) // chunk_size)
        sys.stdout.write(status_text)
        sys.stdout.flush()
#         freqs, amps = calculate_freqs_amps(df, T)
#         c5K, c10K, c15K, c20K = calculate_fft_coeffs(list(df.iloc[:, 0]))
        process_result_list = calculate_fft_coeffs(list(df.iloc[:, 0]))
    
#         avg, std = measures_of_dist(freqs, amps)
        time_remained = df.iloc[-1, 1]

#         csv_file_stats_df = csv_file_stats_df.append({'Average': avg,
#                                                       'STD': std,
#                                                       'Time_Remained': time_remained},
#                                                      ignore_index=True)
#         csv_file_stats_df = csv_file_stats_df.append({'5000': c5K,
#                                                       '10000': c10K,
#                                                       '15000': c15K,
#                                                       '20000': c20K,
#                                                       'Time_Remained': time_remained},
#                                                      ignore_index=True)
#         print('process_result_list + [time_remained]:', process_result_list + [time_remained])
#         print('Before append, csv_file_stats_df:', csv_file_stats_df)
#         print('csv_file_stats_df.columns:', csv_file_stats_df.columns)
        csv_file_stats_df = csv_file_stats_df.append(pd.Series(process_result_list + [time_remained],
                                                               index=csv_file_stats_df.columns),
                                                     ignore_index=True)

#         print('After append, csv_file_stats_df:', csv_file_stats_df)
        
    chunk_stats_file_name = 'chunk_stats_' + csv_file_name + '.pkl'
    csv_file_stats_df.to_pickle(chunk_stats_file_name)
    
# print('-------------------------')
# print()
# print('Completed all files', train_set_csvs)
sys.stdout.write('\rCompleted all files ' + str(train_set_csvs))
sys.stdout.flush()

Completed all files ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']

In [10]:
# results_avg_std_time_remained = pd.DataFrame(columns=['Average', 'STD', 'Time_Remained'])
# results_process_and_time_remained = pd.DataFrame(columns=['5000',
#                                                           '10000',
#                                                           '15000',
#                                                           '20000',
#                                                           '25000',
#                                                           'Time_Remained'])
results_process_and_time_remained = pd.DataFrame(columns=features+['Time_Remained'])
# results_process_and_time_remained = pd.DataFrame()

In [11]:
for csv_file_name in train_set_csvs:
    df_file_name = 'chunk_stats_' + csv_file_name + '.pkl'
    chunk_df = pd.read_pickle(df_file_name)
#     results_avg_std_time_remained = results_avg_std_time_remained.append(chunk_df, ignore_index=True)
    results_process_and_time_remained = results_process_and_time_remained.append(chunk_df,
                                                                                 ignore_index=True,
                                                                                 sort=False)

In [12]:
# results_avg_std_time_remained
results_process_and_time_remained.head()

,5KMax,5KAvg,5KStd,10KMax,10KAvg,10KStd,15KMax,15KAvg,15KStd,20KMax,20KAvg,20KStd,25KMax,25KAvg,25KStd,30KMax,30KAvg,30KStd,Time_Remained
0,732610.0,143.912367,10405.942700,732610.0,70.968719,7477.516039,732610.0,45.750652,6466.661873,732610.0,32.045722,6084.374964,732610.0,26.529029,5538.165517,732610.0,19.088657,5083.794709,1.430797
1,708865.0,133.813010,10136.272301,708865.0,64.531599,7544.161357,708865.0,52.472938,6721.103958,708865.0,44.389343,6640.187394,708865.0,33.431662,6084.912397,708865.0,29.982647,5583.991456,1.391499
2,735959.0,193.665903,10504.359311,735959.0,108.423915,7890.204148,735959.0,68.295336,7004.419740,735959.0,51.984008,6970.667037,735959.0,33.586423,6387.759858,735959.0,21.091974,5863.191283,1.353196
3,735343.0,152.570859,10612.815371,735343.0,77.799647,8164.249430,735343.0,41.854227,7050.315632,735343.0,27.278968,6969.794936,735343.0,19.629223,6366.904849,735343.0,15.943259,5845.388882,1.313798
4,736310.0,129.797777,10508.454710,736310.0,61.205543,8503.581755,736310.0,46.405526,7536.030122,736310.0,46.532470,7186.499562,736310.0,36.138594,6558.105892,736310.0,33.938496,6020.034317,1.274400


In [13]:
# X = results_avg_std_time_remained[['Average', 'STD']]
# X = results_process_and_time_remained[['5000', '10000', '15000', '20000', '25000']]
X = results_process_and_time_remained[features]
# y = results_avg_std_time_remained[['Time_Remained']]
y = results_process_and_time_remained[['Time_Remained']]

In [14]:
lm = linear_model.LinearRegression()
model = lm.fit(X, y)

In [15]:
predictions = lm.predict(X)

In [16]:
import pickle

# Save to file in the current working directory
pkl_filename = "pickle_model_1_to_17.pkl"
# pkl_filename = "pickle_model_1_and_17.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(X, y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(X)

Test score: 14.72 %


## Test the Model on Test Data

In [17]:
test_sets = pd.read_csv('../data/sample_submission.csv')
# test_sets = pd.read_csv('../data/sample_submission_tiny.csv')
total_test_sets = len(test_sets)

In [ ]:
# test_csv_file_name = '../data/tests/' + 'seg_ffe7cc.csv'
# test_set = pd.read_csv(test_csv_file_name)
# test_set.head(5)

In [18]:
evaluation_results_df = pd.DataFrame(columns=['seg_id', 'time_to_failure'])
# T = 1.469099981 - 1.4690999799
for index, row in test_sets.iterrows():
#     print('----------------')
#     print('Evaluating test', str(index) + '/' + str(total_test_sets))
    sys.stdout.write('\rEvaluating test ' + \
                     str(row['seg_id']) + \
                     ' (' + \
                     str(index) + \
                     '/' + \
                     str(total_test_sets) + \
                     ')')
    sys.stdout.flush()
    
    test_csv_file_name = '../data/tests/' + row['seg_id'] + '.csv'
    test_set = pd.read_csv(test_csv_file_name)
    test_set.head(5)
    
    X = test_set[['acoustic_data']]
    
#     freqs, amps = calculate_freqs_amps(X, T)
#     avg, std = measures_of_dist(freqs, amps)
#     c5K, c10K, c15K, c20K, c25K = calculate_fft_coeffs(X.iloc[:, 0])
    process_result_list = calculate_fft_coeffs(X.iloc[:, 0])
#     c5K, c10K, c15K, c20K = calculate_fft_coeffs(X[:, 0])
    
#     Ypredict = pickle_model.predict([[c5K, c10K, c15K, c20K, c25K]])
    Ypredict = pickle_model.predict([process_result_list])
    
    evaluation_results_df = evaluation_results_df.append({'seg_id': row['seg_id'],
                                                          'time_to_failure': Ypredict[0][0]},
                                                          ignore_index=True)

evaluation_results_df.to_csv('../data/result_submission.csv', index=False)
# evaluation_results_df.to_csv('../data/result_submission_tiny.csv', index=False)
# print('Completed writing the results to ../data/result_submission.csv')
sys.stdout.write('\rCompleted writing the results to ../data/result_submission.csv')
sys.stdout.flush()

Completed writing the results to ../data/result_submission.csv

## Experiment with FFT Coefficients

In [ ]:
from scipy.fftpack import fft
# Number of samplepoints
N = 600
# sample spacing
T = 1.0 / 800.0
x = np.linspace(0.0, N*T, N)
y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
yf = fft(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
import matplotlib.pyplot as plt
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.show()

In [ ]:
import scipy

In [ ]:
from scipy.signal import argrelmax

f = xf[scipy.signal.argrelmax(yf[0:N//2])]
Af = np.abs(yf[argrelmax(yf[0:N//2])])

In [ ]:
Af

In [ ]:
x = np.array([1.0, 2.0, 1.0, -1.0, 1.5, 1.0])
fft_x = fft(x)
print('fft_x:', fft_x)

yr = rfft(x)
print('yr:', yr)

In [ ]:
t = np.linspace(0, 0.5, 500)
s = np.sin(10 * 2 * np.pi * t) + 0.5 * np.sin(30 * 2 * np.pi * t)

In [ ]:
plt.plot(t, s)

In [ ]:
len(s)

In [ ]:
rfft_s = rfft(s)

In [ ]:
plt.plot(rfft_s[0:50])

## Experiment with Simple FFT

In [ ]:
t = np.arange(256)
sp = np.fft.fft(np.sin(t))
freq = np.fft.fftfreq(t.shape[-1])
# plt.plot(freq, sp.real, freq, sp.imag)
# plt.plot(freq, sp.real)
plt.plot(freq, abs(sp))

plt.show()

In [ ]:
freq

In [ ]:
len(freq)

In [ ]:
t = np.linspace(0, 0.5, 500)
s = np.sin(40 * 2 * np.pi * t) + 0.5 * np.sin(90 * 2 * np.pi * t)

plt.ylabel("Amplitude")
plt.xlabel("Time [s]")
plt.plot(t, s)
plt.show()

In [ ]:
fft = np.fft.fft(s)
T = t[1] - t[0]  # sampling interval 
N = s.size

# 1/T = frequency
f = np.linspace(0, 1 / T, N)

plt.ylabel("Amplitude")
plt.xlabel("Frequency [Hz]")
plt.bar(f[:N // 2], np.abs(fft)[:N // 2] * 1 / N, width=1.5)  # 1 / N is a normalization factor
plt.show()

In [ ]:
plt.plot(f[:N // 2], np.abs(fft)[:N // 2] * 1 / N)

In [ ]:
len(f)

In [ ]:
len(fft)

In [ ]:
# f

In [ ]:
# fft

In [ ]:
# np.abs(fft)

In [ ]:
# f[:N // 2]

In [ ]:
# np.abs(fft)[:N // 2] * 1 / N

In [ ]:
data = [1, 2, 3, 4]
data

In [ ]:
np.average(data)

In [ ]:
# np.average([1, 2, 3, 4], weights=[10, 1, 1, 1])
np.average(np.abs(fft), weights=f)

In [ ]:
def measures_of_dist(weights, values):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    
    return average, math.sqrt(variance)

In [ ]:
# m, s = measures_of_dist([1, 2, 3, 4], [10, 1, 1, 1])
m, s = measures_of_dist(np.abs(fft), f)

In [ ]:
m

In [ ]:
1.469099981 - 1.4690999799

## Draft

In [ ]:
!ls -l ../data/train/

In [ ]:
# time train_set = pd.read_csv('../data/train/1.csv', skiprows=None, nrows=5656574)
train_set = pd.read_csv('../data/train/1.csv')

In [ ]:
# train_set

In [ ]:
%matplotlib inline

import matplotlib
# import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_set.iloc[4400000:4500000, 0])

In [ ]:
# x = train_set.iloc[0:5000, 0]
x = train_set.iloc[0:150000, 0]

In [ ]:
# x

In [ ]:
f, t, Zxx = signal.stft(x, nperseg=5000)

In [ ]:
plt.figure(figsize=(20,10))
plt.pcolormesh(t, f, np.abs(Zxx), cmap="binary")
plt.colorbar()
plt.title('STFT Magnitude')
plt.ylabel('Frequency')
plt.xlabel('Time')
plt.show()

In [ ]:
rfft_values = rfft(x)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rfft_values[1:25000])

In [ ]:
t

In [ ]:
len(t)

In [ ]:
len(Zxx)

In [ ]:
len(Zxx[0])

In [ ]:
x = np.array([0.0, 1.0, 2.0, 3.0,  4.0,  5.0])
y = np.array([0.0, 0.8, 0.9, 0.1, -0.8, -1.0])
z = np.polyfit(x, y, 3)
z

In [ ]:
import sys

In [ ]:
for i in range(1000):
#     print('i:', i)
    sys.stdout.write("\rDoing thing %i" % i)
    sys.stdout.flush()